In [1]:
import os
import math
import shutil
import numpy as np
from random import seed
from multiprocessing import Pool

from keras.optimizers import *
from keras.losses import *
from keras.callbacks import *
from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
GPUS = '0,1'
RND = 777
RUN = 'E5'
OUT_DIR = 'out'
TENSORBOARD_DIR = '/tensorboard/tf-speech-v2/%s_$fold$' % RUN
MODELS_DIR = '%s/models/run_%s/fold_$fold$' % (OUT_DIR, RUN)
INPUT_SIZE = (96, 96, 1)  # n_mels x width x 1ch
FOLDS = 10

In [3]:
np.random.seed(RND)
seed(RND)

In [4]:
# make predictions dir
os.makedirs('%s/predictions/run_%s'%(OUT_DIR, RUN), exist_ok=True)

In [5]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = GPUS

In [6]:
%run '../data-generator.ipynb'
%run '../models.ipynb'

In [7]:
def choose_batch(n, train_X, train_Y, train_files, val_files):
    assert isinstance(val_files, set)

    # extra random indexes to search for files not in val_files
    def _extra_indexes():
        return np.random.randint(0, len(train_X), size=int(n * 0.15))

    ii = np.random.randint(0, len(train_X), size=n)
    extra_ii = []

    replaced = 0

    # replace indexes with files occuring in val_files
    for j in range(len(ii)):
        if '(silence)' != train_files[ii[j]]:
            while train_files[ii[j]] in val_files:
                if len(extra_ii) == 0: extra_ii = _extra_indexes()
                ii[j], extra_ii = extra_ii[0], extra_ii[1:]
                replaced += 1

    X = train_X[ii]
    Y = train_Y[ii]
    files = train_files[ii]
    
    return X, Y, files

In [8]:
train_X = np.memmap('%s/train_X.mem' % OUT_DIR, np.float32,
                    'r').reshape((-1, ) + INPUT_SIZE)
train_Y = np.memmap('%s/train_Y.mem' % OUT_DIR, np.float32, 'r').reshape(
    (-1, len(LABELS)))

train_files = np.load('%s/train_files.npy' % OUT_DIR)

assert len(train_Y) == len(train_X)
assert len(train_files) == len(train_X)

print('len(train_X):', len(train_X))

len(train_X): 1000000


In [9]:
# training params
N_PER_BATCH = 500
# last number splits train set into XX epochs
STEPS_PER_EPOCH = len(train_X) // N_PER_BATCH // 10
N_EPOCHS = 200

In [ ]:
test_X = np.memmap('%s/test/test_X.mem' % (OUT_DIR), np.float32,
                   'r').reshape((-1, ) + INPUT_SIZE)

In [ ]:
for fold in range(FOLDS):

    print('fold:', fold)

    # read val data
    val_X = np.load('%s/val/val_X_%d.npy' % (OUT_DIR, fold))
    val_Y = np.load('%s/val/val_Y_%d.npy' % (OUT_DIR, fold))
    val_files = np.load('%s/val/val_files_%d.npy' % (OUT_DIR, fold))
    assert len(val_X) == len(val_files)
    assert len(val_Y) == len(val_files)
    print('len(val_X):', len(val_X))
    val_files = set(val_files)

    # create dir to store models
    models_dir = MODELS_DIR.replace('$fold$', str(fold))
    os.makedirs(models_dir, exist_ok=True)
    print('models_dir:', models_dir)

    def train_generator(n_per_batch):
        while True:
            X, Y, files = choose_batch(n_per_batch, train_X, train_Y,
                                       train_files, val_files)
            yield (X, Y)

    # rm/create tensorboard dir
    tensorboard_dir = TENSORBOARD_DIR.replace('$fold$', str(fold))
    shutil.rmtree(tensorboard_dir, ignore_errors=True)
    os.makedirs(tensorboard_dir)
    print('tensorboard_dir:', tensorboard_dir)

    # create model
    model = Model_6(input_size=INPUT_SIZE, output_size=len(LABELS))
    model.build()

    # use x gpus
    
    n_gpus = len(GPUS.split(','))
    
    if n_gpus > 1:
        m = multi_gpu_model(model.m, gpus=n_gpus)
    else:
        m = model.m

    # fit model
    
    optimizer = RMSprop(lr=1e-4)
    m.compile(
        optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy']\
    )

    m.fit_generator(
        train_generator(N_PER_BATCH),
        STEPS_PER_EPOCH,
        epochs=N_EPOCHS,
        validation_data=(val_X, val_Y),
        callbacks=[
            TensorBoard(log_dir=tensorboard_dir),
            #             ModelCheckpoint(
            #                 models_dir +
            #                 '/e{epoch:03d}-l={loss:.5f}-vl={val_loss:.5f}-a={acc:.5f}-va={val_acc:.5f}.h5',
            #                 monitor='val_acc',
            #                 verbose=0,
            #                 save_best_only=True,
            #                 save_weights_only=False,
            #                 mode='auto'),
            ReduceLROnPlateau(
                monitor='val_loss', factor=0.2, patience=3, min_lr=1e-9,
                verbose=1),
            EarlyStopping(
                monitor='val_acc',
                min_delta=0.000001,
                patience=10,
                verbose=1,
                mode='auto')
        ])

    # predict on holdout
    holdout_X = np.load('%s/holdout/holdout_X.npy' % (OUT_DIR))
    holdout_Y = np.load('%s/holdout/holdout_Y.npy' % (OUT_DIR))
    hp = m.predict(holdout_X)
    np.save('%s/predictions/run_%s/holdout_predictions_%d.npy' % (OUT_DIR, RUN,
                                                                  fold), hp)

    # eval on holdout
    print('evaluation on holdout:')
    print(m.metrics_names)
    print(m.evaluate(holdout_X, holdout_Y, verbose=0))

    # predict on test data
    test_predictions = m.predict(test_X, verbose=1, batch_size=100)
    np.save('%s/predictions/run_%s/test_predictions_%d.npy' %
            (OUT_DIR, RUN, fold), test_predictions)

    print('')

fold: 0
len(val_X): 2441
models_dir: out/models/run_E5/fold_0
tensorboard_dir: /tensorboard/tf-speech-v2/E5_0
Epoch 1/200
200/200 [==============================] - 143s 713ms/step - loss: 2.0305 - acc: 0.3833 - val_loss: 1.3150 - val_acc: 0.6178
Epoch 2/200
200/200 [==============================] - 139s 693ms/step - loss: 1.1149 - acc: 0.6236 - val_loss: 0.4954 - val_acc: 0.8476
Epoch 3/200
200/200 [==============================] - 138s 691ms/step - loss: 0.8038 - acc: 0.7362 - val_loss: 0.4401 - val_acc: 0.8578
Epoch 4/200
200/200 [==============================] - 139s 693ms/step - loss: 0.6455 - acc: 0.7902 - val_loss: 0.3071 - val_acc: 0.9095
Epoch 5/200
200/200 [==============================] - 139s 697ms/step - loss: 0.5445 - acc: 0.8245 - val_loss: 0.2843 - val_acc: 0.9123
Epoch 6/200
200/200 [==============================] - 138s 690ms/step - loss: 0.4842 - acc: 0.8457 - val_loss: 0.4571 - val_acc: 0.8660
Epoch 7/200
200/200 [==============================] - 139s 697ms/st

Epoch 21/200
200/200 [==============================] - 138s 691ms/step - loss: 0.0497 - acc: 0.9839 - val_loss: 0.2600 - val_acc: 0.9549
Epoch 22/200
200/200 [==============================] - 138s 691ms/step - loss: 0.0473 - acc: 0.9849 - val_loss: 0.2661 - val_acc: 0.9558
Epoch 23/200
199/200 [============================>.] - ETA: 0s - loss: 0.0450 - acc: 0.9854
Epoch 00023: reducing learning rate to 7.999999979801942e-07.
200/200 [==============================] - 138s 688ms/step - loss: 0.0449 - acc: 0.9854 - val_loss: 0.2668 - val_acc: 0.9529
Epoch 24/200
 48/200 [======>.......................] - ETA: 1:45 - loss: 0.0425 - acc: 0.9865

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



200/200 [==============================] - 138s 692ms/step - loss: 0.0434 - acc: 0.9861 - val_loss: 0.2741 - val_acc: 0.9537
Epoch 29/200
199/200 [============================>.] - ETA: 0s - loss: 0.0437 - acc: 0.9863
Epoch 00029: reducing learning rate to 3.199999980552093e-08.
200/200 [==============================] - 138s 692ms/step - loss: 0.0436 - acc: 0.9863 - val_loss: 0.2743 - val_acc: 0.9541
Epoch 30/200
200/200 [==============================] - 138s 690ms/step - loss: 0.0429 - acc: 0.9863 - val_loss: 0.2745 - val_acc: 0.9541
Epoch 31/200
200/200 [==============================] - 138s 690ms/step - loss: 0.0425 - acc: 0.9863 - val_loss: 0.2736 - val_acc: 0.9541
Epoch 32/200
199/200 [============================>.] - ETA: 0s - loss: 0.0406 - acc: 0.9876
Epoch 00032: reducing learning rate to 6.399999818995639e-09.
200/200 [==============================] - 138s 689ms/step - loss: 0.0407 - acc: 0.9875 - val_loss: 0.2729 - val_acc: 0.9541
Epoch 00032: early stopping
evaluation 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



200/200 [==============================] - 141s 707ms/step - loss: 1.9947 - acc: 0.3868 - val_loss: 1.0441 - val_acc: 0.6639
Epoch 2/200
200/200 [==============================] - 139s 696ms/step - loss: 1.0886 - acc: 0.6346 - val_loss: 0.6402 - val_acc: 0.8012
Epoch 3/200
200/200 [==============================] - 139s 695ms/step - loss: 0.7886 - acc: 0.7423 - val_loss: 0.3473 - val_acc: 0.8881
Epoch 4/200
200/200 [==============================] - 139s 695ms/step - loss: 0.6368 - acc: 0.7937 - val_loss: 0.5664 - val_acc: 0.8238
Epoch 5/200
200/200 [==============================] - 138s 692ms/step - loss: 0.5388 - acc: 0.8265 - val_loss: 0.5397 - val_acc: 0.8369
Epoch 6/200
200/200 [==============================] - 139s 697ms/step - loss: 0.4770 - acc: 0.8477 - val_loss: 0.2131 - val_acc: 0.9365
Epoch 7/200
200/200 [==============================] - 138s 691ms/step - loss: 0.4203 - acc: 0.8664 - val_loss: 0.3296 - val_acc: 0.9066
Epoch 8/200
200/200 [==============================] 

In [ ]:
# .4031 .7079 .8292